# ROOT files
This document deals with ROOT files only (measured at V20 in February 2018). It displays metadata stored in these files and create .png files of all plots (1D and 2D).  

For the 2D plots, there is the option of inverting the `y-axis`.

**Information about data in ROOT files**

| Spectrum     | Folders with data to plot | Sample   | Duration |
| :----------- |:--------------------------| :------- | :------- |
| Spectrum3    | Meas_3;1                  | NAK      | 13 hours |
| Spectrum8    | Meas_2;1                  | Si-NIST  | 19 hours |
| Spectrum9    | Meas_1;1                  | Y2O3     | 22 hours |
| Spectrum11   | Meas_1;1                  | NAK      | 22 hours | 
| Spectrum12   | Meas_1;1                  | NAK      | 22 hours |

In [ ]:
import ROOT
import matplotlib.pyplot as plt
import numpy as np

import ipywidgets

# to remove problems with too many figures open
plt.rcParams.update({'figure.max_open_warning': 0})

# Display metadata stored in a ROOT file
The information related to only one ROOT file is displayed. 


In [ ]:
path_to_file = '/Users/celinedurniak/Documents/test_root/Diffraction/TBL_Data_DreamTeam_Feb2018/DREAMTeam_Feb2018/DENEX/'

# Create dictionary to generate plots
# Each entry corresponds to the spectrum number, the associated root file and the folder to get the data from
dict_root_files = {'Spectrum03': ("Spectrum03_DENEX006_1_18-02-05_0000.root", "Meas_3"), 
                   'Spectrum08': ("Spectrum08_DENEX006_1_18-02-07_0002.root", "Meas_2"),
                   'Spectrum09': ("Spectrum09_DENEX006_1_18-02-08_0001.root", "Meas_1"),
                   'Spectrum11': ("Spectrum11_DENEX006_1_18-02-09_0001.root", "Meas_1"),
                   'Spectrum12': ("Spectrum12_DENEX006_1_18-02-10_0000.root", "Meas_1")}

In [ ]:
# Display metadata stored in selected ROOT file
ROOTfile,  dir_with_data = dict_root_files['Spectrum03']

myFile = ROOT.TFile.Open(path_to_file + ROOTfile, "read")

for keyName in myFile.GetListOfKeys():
    myObject = myFile.Get(keyName.GetName())
    for key in myObject.GetListOfKeys():
        if "TH1" in key.GetClassName() and 'BoardParam_run_1' in key.GetName():
            mySubObject = myObject.Get(key.GetName())
            axis_x = mySubObject.GetXaxis()
        
            nb_xbins = axis_x.GetNbins()
            for i in range(1, nb_xbins+1):
                print(f"{axis_x.GetBinLabel(i)}: {mySubObject.GetBinContent(i)}")
myFile.Close()       

# Plot all graphs stored in ROOT files and save as .png files
This process is done for ROOT files related to Spectrum03, 08, 09, 11, and 12.

In [ ]:
invert_y = ipywidgets.Checkbox(
    value=True,
    description='Invert y-axis of 2D plots for ROOT files?')
invert_y

In [ ]:
# remove spectrum09 from dictionary. It will be dealt with in the next cell
if 'Spectrum09' in dict_root_files.keys():
    del dict_root_files['Spectrum09']

# loop over selected root files

for key_spec in dict_root_files.keys():
    print(50*'-','\n',key_spec)
    
    ROOTfile = dict_root_files[key_spec][0]
    dir_with_data = dict_root_files[key_spec][1]

    myFile = ROOT.TFile.Open(path_to_file + ROOTfile, "read")

    for keyName in myFile.GetListOfKeys():
        myObject = myFile.Get(keyName.GetName()) 
    
        if dir_with_data in str(myObject):
            for key in myObject.GetListOfKeys():
                # 1D line plot
                if "TH1I" in key.GetClassName():     
                    mySubObject = myObject.Get(key.GetName())  
                    
                    # extract data
                    axis_x = mySubObject.GetXaxis()
                    
                    ar_object = np.zeros(axis_x.GetNbins()-1)
                    for i in range(1, axis_x.GetNbins()):
                        ar_object[i-1]=mySubObject.GetBinContent(i)
                
                    # plot
                    fig, ax = plt.subplots()
                    ax.plot(ar_object)
                    ax.set_title(key.GetName())
                    plt.show()
                    
                    # to save plot uncomment the next line
                    # plt.savefig(ROOTfile[:10] + key.GetName().replace(',','_') + '.png')
                    
                # 2D contourplot
                elif "TH2" in key.GetClassName():
                     
                    mySubObject = myObject.Get(key.GetName())
                   
                    # extract data
                    axis_x = mySubObject.GetXaxis()
                    axis_y = mySubObject.GetYaxis()
                
                    # create x- and y-axis
                    xaxis = axis_x.GetXmin() \
                        + (axis_x.GetXmax() - axis_x.GetXmin())/(axis_x.GetNbins()-2)*np.arange(axis_x.GetNbins()-1)
                    yaxis = axis_y.GetXmin() \
                        + (axis_y.GetXmax() - axis_y.GetXmin()) / (axis_y.GetNbins() - 2) * np.arange(axis_y.GetNbins()-1)

                    # fill 2d matrice
                    ar_object = np.zeros((axis_x.GetNbins()-1, axis_y.GetNbins()-1))
                    
                    # invert y axis
                    if invert_y.value:
                        for i in range(1, axis_x.GetNbins()):
                            for j in range(1, axis_y.GetNbins()):
                                ar_object[i - 1, axis_y.GetNbins() - j - 1] = mySubObject.GetBinContent(i, j)
                    # keep original orientation of y-axis
                    else:
                        for i in range(1, axis_x.GetNbins()):
                            for j in range(1, axis_y.GetNbins()):
                                ar_object[i-1, j-1] = mySubObject.GetBinContent(i, j)
                        
                    # plot
                    fig, ax = plt.subplots()
                    CS = ax.contourf(xaxis, yaxis, ar_object.transpose(), cmap=plt.cm.get_cmap('gist_earth'))
                    ax.set_title(key.GetName())
                    cbar = fig.colorbar(CS)
                    plt.show()
                
                    # to save plot uncomment the next line
                    # plt.savefig(ROOTfile[:10] + key.GetName().replace(',','_') + '.png')
                  
    myFile.Close()

In [ ]:
# Specific handling of Spectrum09: issue with one dataset
    
ROOTfile = "Spectrum09_DENEX006_1_18-02-08_0001.root"
dir_with_data = "Meas_1"

myFile = ROOT.TFile.Open(path_to_file + ROOTfile, "read")

for keyName in myFile.GetListOfKeys():
    myObject = myFile.Get(keyName.GetName()) 
    
    if dir_with_data in str(myObject):
       
        for key in myObject.GetListOfKeys():
            
            # 1D line plot - discard plotting buggy data
            if "TH1I" in key.GetClassName() and not '1D3' in key.GetName():  
                mySubObject = myObject.Get(key.GetName())
                
                # extract data
                axis_x = mySubObject.GetXaxis()
                    
                ar_object = np.zeros(axis_x.GetNbins()-1)
                for i in range(1, axis_x.GetNbins()):
                    ar_object[i-1]=mySubObject.GetBinContent(i)
                
                # plot
                fig, ax = plt.subplots()
                ax.plot(ar_object)
                ax.set_title(key.GetName())
            
                # to save plot uncomment the next line
                # plt.savefig(ROOTfile[:10] + key.GetName().replace(',','_') + '.png')
                    
            # 2D contourplot
            elif "TH2" in key.GetClassName():
                     
                mySubObject = myObject.Get(key.GetName())
                  
                # extract data
                axis_x = mySubObject.GetXaxis()
                axis_y = mySubObject.GetYaxis()
                
                # create x- and y-axis
                xaxis = axis_x.GetXmin() \
                    + (axis_x.GetXmax() - axis_x.GetXmin())/(axis_x.GetNbins()-2)*np.arange(axis_x.GetNbins()-1)
                yaxis = axis_y.GetXmin() \
                    + (axis_y.GetXmax() - axis_y.GetXmin()) / (axis_y.GetNbins() - 2) * np.arange(axis_y.GetNbins()-1)

                # fill 2d matrice
                ar_object = np.zeros((axis_x.GetNbins()-1, axis_y.GetNbins()-1))

                # invert y axis
                if invert_y.value:
                    for i in range(1, axis_x.GetNbins()):
                        for j in range(1, axis_y.GetNbins()):
                            ar_object[i - 1, axis_y.GetNbins() - j - 1] = mySubObject.GetBinContent(i, j)
                # keep original orientation of y-axis
                else:
                    for i in range(1, axis_x.GetNbins()):
                        for j in range(1, axis_y.GetNbins()):
                            ar_object[i-1, j-1] = mySubObject.GetBinContent(i, j)
                           
                # plot
                fig, ax = plt.subplots()
                CS = ax.contourf(xaxis, yaxis, ar_object.transpose(), cmap=plt.cm.get_cmap('gist_earth'))
                ax.set_title(key.GetName())
                cbar = fig.colorbar(CS)
                
                # to save plot uncomment the next line
                # plt.savefig(ROOTfile[:10] + key.GetName().replace(',','_') + '.png')
                         
myFile.Close()